# Paso a parquet los csv de https://www.kaggle.com/datasets/mkechinov/ecommerce-events-history-in-cosmetics-shop

In [8]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf

conf = (SparkConf().set("spark.executor.memory", "6g")   #<--- Por executor. Como hay dos instancias, la mitad por executor.
                    .set("spark.executor.cores", "3")    #<--- Por cada executor. Como hay 4, 4*3=12 cores se usan.
                    .set("spark.executor.instances", "2")
                    .set("spark.eventLog.enabled", "true"))
spark = SparkSession.builder.master("spark://spark-master:7077").appName("Ecommerce").config(conf=conf).getOrCreate()
spark

In [11]:
%%time
from pyspark.sql.types import StructType
from pyspark.sql.types import StringType,BooleanType,DateType,IntegerType
from pyspark.sql.functions import substring
import pyspark.sql.functions as F

df = spark.read.csv("./datos/ecommerce/*.csv", sep=",", header=True, inferSchema=True)
# shape:
print(80*'=','\nfilas --> ', df.count(), 'columnas --> ', len(df.columns), '\n', 80*'=')
df.show(truncate=False)

df = df.withColumn("proc", substring("event_time",1,10))
df.write.mode("append").partitionBy("proc").parquet("./datos/ecommerce/ecommerce.parquet")

filas -->  20692840 columnas -->  9 
+-----------------------+----------------+----------+-------------------+-------------+---------+-----+---------+------------------------------------+
|event_time             |event_type      |product_id|category_id        |category_code|brand    |price|user_id  |user_session                        |
+-----------------------+----------------+----------+-------------------+-------------+---------+-----+---------+------------------------------------+
|2019-12-01 00:00:00 UTC|remove_from_cart|5712790   |1487580005268456287|null         |f.o.x    |6.27 |576802932|51d85cb0-897f-48d2-918b-ad63965c12dc|
|2019-12-01 00:00:00 UTC|view            |5764655   |1487580005411062629|null         |cnd      |29.05|412120092|8adff31e-2051-4894-9758-224bfa8aec18|
|2019-12-01 00:00:02 UTC|cart            |4958      |1487580009471148064|null         |runail   |1.19 |494077766|c99a50e8-2fac-4c4d-89ec-41c05f114554|
|2019-12-01 00:00:05 UTC|view            |5848413   |1487

CPU times: user 14.4 ms, sys: 10.4 ms, total: 24.9 ms
Wall time: 21.9 s


In [ ]:
%%time
spark.sparkContext.stop()
